In [60]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.metrics import accuracy_score, balanced_accuracy_score,precision_score, f1_score, roc_auc_score, roc_curve, precision_recall_curve, auc, recall_score, confusion_matrix,matthews_corrcoef
import sys 
import pickle

from imblearn.under_sampling import RandomUnderSampler
sys.path.append('/Users/jayceepang/msse/capstone/atom2024/atom2024/notebooks/')
from RF_atomver import *
from VisUtils import *

In [61]:
data_dir = '/Users/jayceepang/msse/capstone/atom2024/atom2024/notebooks/NEK/'

excluded_dir = '/Users/jayceepang/msse/capstone/atom2024/atom2024/notebooks/NEK/undersampler_validation/idea4_dir/excluded_inactives/'
dir3 ='/Users/jayceepang/msse/capstone/atom2024/atom2024/notebooks/NEK/undersampler_validation/idea3_dir/same_actives_diff_inactive_train/' 
neks = ['2','3','5','9']

for i, nek in enumerate(neks):
    if nek in ['2','9']: 
        bind_inhib = ['binding', 'inhibition']
    else: 
        bind_inhib = ['binding'] 
    for bi in bind_inhib: 
        if bi == 'binding': 
            this_bi = 'bind'
        else: 
            this_bi = 'inhib' 
        print(f'NEK{nek} {bi}')
        data_dir = f'/Users/jayceepang/msse/capstone/atom2024/atom2024/notebooks/NEK/NEK{nek}/{this_bi}/'
        under = pd.read_csv(f'{data_dir}NEK{nek}_{bi}_moe_UNDER_df.csv') 
        under_actives = under[(under['subset'] == 'train')&(under['active'] == 1)] 
        num_actives = under_actives.shape[0]
        inactive_not_in_UNDERtrain = pd.read_csv(f'{excluded_dir}NEK{nek}_{bi}_moe_excluded_by_RUS.csv')
        new_inactives = inactive_not_in_UNDERtrain.iloc[num_actives:]
        new_under_train = pd.concat([new_inactives, under_actives])
        new_under_train.to_csv(f'{dir3}NEK{nek}_{bi}_moe_UNDER_new_train_inactives.csv')
        

NEK2 binding
NEK2 inhibition
NEK3 binding
NEK5 binding
NEK9 binding
NEK9 inhibition


In [35]:
def add_cm2(df): 
    true_labels = df['y'] 
    predictions = df['prediction']
    cm = confusion_matrix(true_labels, predictions )
    cm_flattened = cm.flatten().tolist()
    df['cm'] = [cm_flattened]* len(df)
    df['prediction_type'] = df.apply(lambda x: prediction_type(x['y'], x['prediction']), axis=1)
    
    df['f1'] = f1_score(df['y'], df['prediction'])
    # df['ROC-AUC'] = roc_auc_score(df['y'], df['prediction'])
    df['ROC-AUC'] ='NA'
    df['MCC'] = matthews_corrcoef(df['y'], df['prediction'])
    df['Balanced Accuracy'] = balanced_accuracy_score(df['y'], df['prediction'])
    return df

    

In [38]:
drop_cols = ['subset', 'compound_id', 'base_rdkit_smiles', 'fold', 'active'] 
save_path ='/Users/jayceepang/msse/capstone/atom2024/atom2024/notebooks/NEK/undersampler_validation/idea3_dir/same_actives_diff_inactive_train/idea3_models_results/'
for i, nek in enumerate(neks):
    if nek in ['2','9']: 
        bind_inhib = ['binding', 'inhibition']
    else: 
        bind_inhib = ['binding'] 
    for bi in bind_inhib: 
        if bi == 'binding': 
            this_bi = 'bind'
        else: 
            this_bi = 'inhib' 
        print(f'NEK{nek} {bi}')
        RF_type = 'RF'
        df = pd.read_csv(f'{dir3}NEK{nek}_{bi}_moe_UNDER_new_train_inactives.csv')
        trainX = df[df['subset'] == 'train'] 
        trainX = trainX.drop(columns='Unnamed: 0')
        trainX = trainX.drop(columns=drop_cols).to_numpy()
       
        train_y = df[df['subset'] == 'train'] 
        train_y = train_y['active'].to_numpy() 

        reg_dir = f'/Users/jayceepang/msse/capstone/atom2024/atom2024/notebooks/NEK/NEK{nek}/{this_bi}/'
        original_df = pd.read_csv(f'{reg_dir}NEK{nek}_{bi}_moe_UNDER_df.csv')
        og_test = original_df[original_df['subset'] =='test'] 
        test_y = og_test['active'].to_numpy()
        testX = og_test.drop(columns=drop_cols).to_numpy()

        model = rf_models(trainX, train_y, testX, test_y, RF_type, {})
        train_df = gather_rf_results(model, trainX, train_y)
        test_df = gather_rf_results(model, testX, test_y)
        train_df['subset'] = 'train' 
        test_df['subset'] = 'test' 
        model_name = f'NEK{nek}_{bi}_moe_UNDER_RF_excluded_train' 
        train_df['model'] = model_name
        
        test_df['model'] = model_name
        train_df = add_cm2(train_df)
        test_df = add_cm2(test_df)
        
        train_df['NEK'] = f'NEK{nek}_{bi}'
        train_df['feat_type'] = 'moe'
        train_df['strategy'] = 'UNDER'
        train_df['RF_type'] = RF_type
        test_df['NEK'] = f'NEK{nek}_{bi}'
        test_df['feat_type'] = 'moe'
        test_df['strategy'] = 'UNDER'
        test_df['RF_type'] = RF_type
        
        with open(f'{save_path}{model_name}.pkl', 'wb') as f: 
            pickle.dump(model,f)

        train_df.to_csv(f'{save_path}{model_name}_train_results.csv',index=False) 
        test_df.to_csv(f'{save_path}{model_name}_test_results.csv',index=False) 
        
        

NEK2 binding
TRAIN: accuracy: 1.000, precision: 1.000, recall: 1.000, specificity: 1.000
TEST: accuracy: 0.898, precision: 0.000, recall: 0.000, specificity: 0.937
NEK2 inhibition
TRAIN: accuracy: 1.000, precision: 1.000, recall: 1.000, specificity: 1.000
TEST: accuracy: 0.944, precision: 0.586, recall: 0.607, specificity: 0.968
NEK3 binding
TRAIN: accuracy: 1.000, precision: 1.000, recall: 1.000, specificity: 1.000
TEST: accuracy: 0.858, precision: 0.103, recall: 0.176, specificity: 0.902
NEK5 binding
TRAIN: accuracy: 1.000, precision: 1.000, recall: 1.000, specificity: 1.000
TEST: accuracy: 0.883, precision: 0.320, recall: 0.400, specificity: 0.925
NEK9 binding
TRAIN: accuracy: 1.000, precision: 1.000, recall: 1.000, specificity: 1.000
TEST: accuracy: 0.905, precision: 0.000, recall: 0.000, specificity: 0.948
NEK9 inhibition
TRAIN: accuracy: 1.000, precision: 1.000, recall: 1.000, specificity: 1.000
TEST: accuracy: 0.938, precision: 0.833, recall: 0.556, specificity: 0.986


In [39]:
results = [] 
metric_cols = ['accuracy', 'precision', 'recall', 'specificity','TN', 'FN', 'FP', 'TP','f1', 'ROC-AUC', 'MCC', 'Balanced Accuracy',
       'model', 'cm', 'prediction_type', 'NEK', 'feat_type', 'strategy','RF_type']

for i, nek in enumerate(neks):
    if nek in ['2','9']: 
        bind_inhib = ['binding', 'inhibition']
    else: 
        bind_inhib = ['binding'] 
    for bi in bind_inhib: 
        if bi == 'binding': 
            this_bi = 'bind'
        else: 
            this_bi = 'inhib' 
        result_df = pd.read_csv(f'{save_path}NEK{nek}_{bi}_moe_UNDER_RF_excluded_train_test_results.csv')
        results.append(result_df.iloc[[0]][metric_cols].values.flatten())
results_df =  pd.DataFrame(results,columns=metric_cols)
results_df['model'] = results_df['model'].str.replace('scaled', 'raw')
results_df['strategy'] =results_df['strategy'].str.replace('scaled', 'raw')
results_df['strategy'] = results_df['strategy'].str.replace('scaled', 'raw')
results_df.to_csv(save_path+'RF_results_newly_trained_with_inactives_excluded_by_RUS.csv', index=False) 
results_df['modeling_type'] = 'RF' 
results_df['set'] = 'train_fold2to5_RUS_excluded'

In [71]:
original_results = pd.read_excel('/Users/jayceepang/msse/capstone/atom2024/atom2024/notebooks/NEK/general_NEK/GP_GPmatern_RF_RFGS_results_all_sets.xlsx')
original_results = original_results[(original_results['set']=='original') & (original_results['modeling_type'] == 'RF') & (original_results['feat_type'] == 'moe')]
only_under = original_results[(original_results['strategy'] == 'UNDER') &(original_results['RF_type'] == 'RF')]
all_under_rf = pd.concat([results_df, only_under])
comparison = all_under_rf[['NEK', 'set','cm', 'recall', 'specificity']]
comparison = comparison.sort_values('NEK')
comparison 

,NEK,set,cm,recall,specificity
0,NEK2_binding,train_fold2to5_RUS_excluded,"[254, 17, 12, 0]",0.000000,0.937269
156,NEK2_binding,original,"[191, 80, 6, 6]",0.500000,0.704797
1,NEK2_inhibition,train_fold2to5_RUS_excluded,"[368, 12, 11, 17]",0.607143,0.968421
188,NEK2_inhibition,original,"[302, 78, 4, 24]",0.857143,0.794737
2,NEK3_binding,train_fold2to5_RUS_excluded,"[239, 26, 14, 3]",0.176471,0.901887
220,NEK3_binding,original,"[135, 130, 3, 14]",0.823529,0.509434
3,NEK5_binding,train_fold2to5_RUS_excluded,"[211, 17, 12, 8]",0.400000,0.925439
252,NEK5_binding,original,"[161, 67, 3, 17]",0.850000,0.706140
4,NEK9_binding,train_fold2to5_RUS_excluded,"[256, 14, 13, 0]",0.000000,0.948148
284,NEK9_binding,original,"[177, 93, 4, 9]",0.692308,0.655556


In [70]:
all_us =  pd.read_excel('/Users/jayceepang/msse/capstone/atom2024/atom2024/notebooks/NEK/general_NEK/GP_GPmatern_RF_RFGS_results_all_sets.xlsx')

all_us  = all_us[(all_us ['set'] !='literature') & (all_us['modeling_type']!='RF_GS')]

all_us =all_us[(all_us['strategy'] != 'SMOTE') & (all_us['set'] !='literature') & (all_us['strategy'] !='ADASYN')]

rf= all_us[all_us['modeling_type'] != 'GP']
rf= all_us[all_us['modeling_type'] != 'GP_matern']

rf_raw = rf[(rf['strategy'] == 'raw') & (rf['RF_type'].isin(['RF', 'RF_BCW', 'BRFC', 'BRFC_BCW']))]
rf_other = rf[(rf['RF_type'] == 'RF') & (rf['strategy'].isin(['UNDER']))]  
rf_us = pd.concat([rf_raw, rf_other])
rf_us_moe = rf_us[rf_us['feat_type'] == 'moe']
make_comparison_us = pd.concat([rf_us_moe, results_df])
make_comparison_us = make_comparison_us.sort_values(['NEK', 'strategy'])
make_comparison_us 
new_order = ['model', 'NEK', 'set', 'RF_type','modeling_type', 'feat_type', 'strategy', 'cm',
       'recall', 'specificity','accuracy', 'precision',  'f1', 'MCC',
       'Balanced Accuracy', 'ROC-AUC', 'TN', 'FN', 'FP', 'TP',
       'prediction_type']
make_comparison_us=make_comparison_us[new_order]
make_comparison_us.to_excel('comparison_us_fold2to5_to_fold2to5diffinactives.xlsx',index=False)

In [67]:
make_comparison_us.columns

Index(['model', 'NEK', 'set', 'modeling_type', 'feat_type', 'strategy', 'cm',
       'recall', 'accuracy', 'precision', 'specificity', 'f1', 'MCC',
       'Balanced Accuracy', 'ROC-AUC', 'RF_type', 'TN', 'FN', 'FP', 'TP',
       'prediction_type'],
      dtype='object')

In [55]:
model	NEK	set	strategy	cm	recall	accuracy	precision	specificity	f1	MCC	Balanced Accuracy	ROC-AUC	RF_type	TN	FN	FP	TP	prediction_type

,model,NEK,set,modeling_type,feat_type,strategy,cm,recall,accuracy,precision,...,f1,MCC,Balanced Accuracy,ROC-AUC,RF_type,TN,FN,FP,TP,prediction_type
152,NEK2_binding_moe_raw_RF,NEK2_binding,original,RF,moe,raw,"[271, 0, 12, 0]",0.000000,0.957597,0.000000,...,0.000000,0.000000,0.500000,0.500000,RF,NaN,NaN,NaN,NaN,NaN
153,NEK2_binding_moe_raw_RF_BCW,NEK2_binding,original,RF,moe,raw,"[271, 0, 11, 1]",0.083333,0.961131,1.000000,...,0.153846,0.282989,0.541667,0.541667,RF_BCW,NaN,NaN,NaN,NaN,NaN
154,NEK2_binding_moe_raw_BRFC,NEK2_binding,original,RF,moe,raw,"[204, 67, 5, 7]",0.583333,0.745583,0.094595,...,0.162791,0.154119,0.668050,0.668050,BRFC,NaN,NaN,NaN,NaN,NaN
155,NEK2_binding_moe_raw_BRFC_BCW,NEK2_binding,original,RF,moe,raw,"[203, 68, 5, 7]",0.583333,0.742049,0.093333,...,0.160920,0.151771,0.666205,0.666205,BRFC_BCW,NaN,NaN,NaN,NaN,NaN
184,NEK2_inhibition_moe_raw_RF,NEK2_inhibition,original,RF,moe,raw,"[372, 8, 13, 15]",0.535714,0.948529,0.652174,...,0.588235,0.564155,0.757331,0.757331,RF,NaN,NaN,NaN,NaN,NaN
185,NEK2_inhibition_moe_raw_RF_BCW,NEK2_inhibition,original,RF,moe,raw,"[372, 8, 14, 14]",0.500000,0.946078,0.636364,...,0.560000,0.536110,0.739474,0.739474,RF_BCW,NaN,NaN,NaN,NaN,NaN
186,NEK2_inhibition_moe_raw_BRFC,NEK2_inhibition,original,RF,moe,raw,"[371, 9, 14, 14]",0.500000,0.943627,0.608696,...,0.549020,0.522121,0.738158,0.738158,BRFC,NaN,NaN,NaN,NaN,NaN
187,NEK2_inhibition_moe_raw_BRFC_BCW,NEK2_inhibition,original,RF,moe,raw,"[371, 9, 14, 14]",0.500000,0.943627,0.608696,...,0.549020,0.522121,0.738158,0.738158,BRFC_BCW,NaN,NaN,NaN,NaN,NaN
216,NEK3_binding_moe_raw_RF,NEK3_binding,original,RF,moe,raw,"[262, 3, 17, 0]",0.000000,0.929078,0.000000,...,0.000000,-0.026264,0.494340,0.494340,RF,NaN,NaN,NaN,NaN,NaN
217,NEK3_binding_moe_raw_RF_BCW,NEK3_binding,original,RF,moe,raw,"[265, 0, 16, 1]",0.058824,0.943262,1.000000,...,0.111111,0.235530,0.529412,0.529412,RF_BCW,NaN,NaN,NaN,NaN,NaN
